In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('device_failure.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [4]:
len(df['device'])

124494

In [5]:
# Most rows duplicate devices
df['device'].duplicated().sum()

123325

In [6]:
# How many unique devices are there?
df['device'].nunique()

1169

In [7]:
# There is only one instance where the same device has two measurements on the same day.
df[['device', 'date']].duplicated().sum()

1

In [8]:
# what is that row?
df[df[['device', 'date']].duplicated()==True]

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
101335,2015-07-10,S1F0R4Q8,0,192721392,0,0,0,8,213700,0,0,0


In [9]:
# we should remove one of these duplicate rows.
df.loc[(df['device']=='S1F0R4Q8') & (df['date']=='2015-07-10')]

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
101334,2015-07-10,S1F0R4Q8,0,192721392,0,0,0,8,213700,0,0,0
101335,2015-07-10,S1F0R4Q8,0,192721392,0,0,0,8,213700,0,0,0


In [10]:
# There is no missing data anywhere.
df.isnull().sum()

date          0
device        0
failure       0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
dtype: int64

In [19]:
# Are attributes 7 and 8 identical? Yes they are.
False in list(df['attribute7']==df['attribute8'])

False

In [20]:
# All devices start on the same day: January 1st, 2015
df['date'].describe()

count         124494
unique           304
top       2015-01-01
freq            1163
Name: date, dtype: object

In [28]:
# Example of a device
'S1F11MB0' in list(df['device'].value_counts().index)

True

In [29]:
df.loc[df['device']=='S1F11MB0']

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
443,2015-01-01,S1F11MB0,0,173113976,0,0,0,5,198441,0,0,0
1606,2015-01-02,S1F11MB0,0,120647592,0,0,0,5,198441,0,0,0
2769,2015-01-03,S1F11MB0,0,100757160,0,0,0,5,198441,0,0,0
3932,2015-01-04,S1F11MB0,0,192750960,0,0,0,5,198441,0,0,0
5093,2015-01-05,S1F11MB0,0,211589728,0,0,0,5,198441,0,0,0
6213,2015-01-06,S1F11MB0,0,159348880,0,0,0,5,199492,0,0,0
23140,2015-01-29,S1F11MB0,1,92061968,0,0,0,11,211476,0,0,0
